In [1]:
import dask
dask.config.set({"dataframe.query-planning": False})

import scanpy as sc
import scipy
import numpy as np
import pandas as pd
import sys
import matplotlib.patches as mpatches
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.append("../../../workflow/scripts/")
import _utils
import readwrite
cfg = readwrite.config()

/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/importlib/__init__.py:126: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  return _bootstrap._gcd_import(name[level:], package, level)
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/cudf/utils/_ptxcompiler.py:64: UserWarning: Error getting driver and runtime versions:

stdout:



stderr:

Traceback (most recent call last):
  File "<string>", line 4, in <module>
  File "/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/numba_cuda/numba/cuda/cudadrv/driver.py", line 272, in __getattr__
    raise CudaSupportError("Error at driver init: \n%s:" %
numba.cuda.cudadrv.error.CudaSupportError: Error a

## Compute metrics diffexpr logreg

In [8]:
segmentation = '10x_15um'
condition = 'breast'
panel = 'breast'
donor = '1GUC'
sample = '1GUC'
k = (segmentation,condition,panel,donor,sample)
if 'proseg' in segmentation:
    k_dir = ('proseg',condition,panel,donor,sample)
else:
    k_dir = k

name = '/'.join(k)
name_dir = '/'.join(k_dir)
# sample_dir = Path(f'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/{name_dir}') / 'raw_results'
sample_dir = Path(f'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/{name}') / 'normalised_results/outs'
sample_counts = Path(f'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/std_seurat_analysis/{name}/lognorm/normalised_counts/data.parquet')
sample_idx = Path(f'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/std_seurat_analysis/{name}/lognorm/normalised_counts/cells.parquet')
sample_annotation = f'/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/cell_type_annotation/{name}/lognorm/reference_based/matched_reference_combo/rctd_class_aware/Level2.1/single_cell/labels.parquet'

out_file_df_permutations = sample_dir / 'permutation_summary.parquet'
out_file_df_importances = sample_dir / 'importances.parquet'
out_file_df_diffexpr = sample_dir / 'diffexpr.parquet'
out_file_df_markers_rank_significance_logreg = sample_dir / 'markers_rank_significance_logreg.json'
out_file_df_markers_rank_significance_diffexpr = sample_dir / 'markers_rank_significance_diffexpr.json'
# out_dir_liana_lrdata = sample_dir / 'liana_lrdata_folder'

level = 'Level2.1'
radius = 10
n_permutations = 30
n_splits= 5
n_repeats = 5
top_n = 20
top_n_lr = 10
cti = "macrophage"
ctj = "malignant cell"
scoring = 'f1'
cv_mode = 'spatial'
# markers = '/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/markers/Xenium_panels_common_markers.csv'
markers = 'diffexpr'
precomputed_adata_obs = None
precomputed_ctj_markers = None

level_simplified = 'Level1'
palette = pd.read_csv('/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/metadata/col_palette_cell_types_combo.csv')
cell_types_mapping = palette.set_index('Level2.1')[level_simplified]
cell_types_mapping[cell_types_mapping.str.contains('malignant')] = 'malignant cell'

list_n_markers = [10, 20, 30, 40, 50]
obsm = "spatial"
label_key = "label_key"
index_diffexpr_metrics = [
    "Name",
    "Term",
    "ES",
    "NES",
    "NOM p-val",
    "FDR q-val",
    "FWER p-val",
    "Tag %",
    "Gene %",
    "Lead_genes",
    "hypergeometric_pvalue",
    f"n_hits_top_n={top_n}",
    "mean_zscore",
    "mean_zscore_pvalue",
]



####
#### READ DATA
####
# read raw data to get spatial coordinates
adata = readwrite.read_xenium_sample(
    sample_dir,
    cells_as_circles=False,
    cells_boundaries=False,
    cells_boundaries_layers=False,
    nucleus_boundaries=False,
    cells_labels=False,
    nucleus_labels=False,
    transcripts=False,
    morphology_mip=False,
    morphology_focus=False,
    aligned_images=False,
    anndata=True,
)
if 'proseg_expected' in sample_counts.as_posix():
    adata.obs_names = 'proseg-'+adata.obs_names.astype(str)


# read normalised data, filter cells
X_normalised = pd.read_parquet(sample_counts)
X_normalised.index = pd.read_parquet(sample_idx).iloc[:, 0]
X_normalised.columns = X_normalised.columns.str.replace('.','-')
adata = adata[X_normalised.index,X_normalised.columns]
adata.layers['X_normalised'] = X_normalised

# read labels
label_key = "label_key"
adata.obs[label_key] = pd.read_parquet(sample_annotation).set_index("cell_id").iloc[:, 0]
adata = adata[adata.obs[label_key].notna()]  # remove NaN annotation

if "Level2.1" in sample_annotation:
    # for custom Level2.1, simplify subtypes
    adata.obs.loc[adata.obs[label_key].str.contains("malignant"), label_key] = "malignant cell"
    adata.obs.loc[adata.obs[label_key].str.contains("T cell"), label_key] = "T cell"

# read markers if needed
if markers != "diffexpr":
    if "Level2.1" in sample_annotation and markers == "common_markers":
        # custom mapping for Level2.1: simplify to Level1 to assess with known markers
        level_simplified = "Level1"

        df_markers = pd.read_csv("data/markers/Xenium_panels_common_markers.csv")[["cell_type", "gene_name"]]
        palette = pd.read_csv("data/xenium/metadata/col_palette_cell_types_combo.csv")

        cell_types_mapping = palette.set_index("Level2.1")[level_simplified].replace(r" of .+", "", regex=True)
        cell_types_mapping[cell_types_mapping.str.contains("malignant")] = "malignant cell"
        adata.obs[label_key] = adata.obs[label_key].replace(cell_types_mapping)

    else:
        df_markers = pd.read_csv(markers)[["cell_type", "gene_name"]]

    ct_not_found = adata.obs[label_key][~adata.obs[label_key].isin(df_markers["cell_type"])].unique()
    print(f"Could not find {ct_not_found} in markers file")
    adata = adata[adata.obs[label_key].isin(df_markers["cell_type"])]
else:
    # get precomputed markers from raw data
    if precomputed_ctj_markers is not None:
        print("Loading precomputed markers")
        df_ctj_marker_genes_precomputed = pd.read_parquet(precomputed_ctj_markers)

####
#### PREPARE DATA
####
# log-normalize before DE
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)


# define target (cell type j presence in kNN)
if precomputed_adata_obs is not None:
    print("Loading precomputed adata obs. Replacing loaded labels")
    adata.obs = pd.read_parquet(precomputed_adata_obs).loc[adata.obs_names]
else:
    knnlabels, knndis, knnidx, knn_graph = _utils.get_knn_labels(
        adata, radius=radius, label_key=label_key, obsm=obsm, return_sparse_neighbors=True
    )

    adata.obsp[f"{obsm}_connectivities"] = knn_graph


# iterate over targets permutations (cell type i with cell type j presence in kNN)
u_cell_types = adata.obs[label_key].unique()
df_ctj_marker_genes = {}
df_permutations_logreg = {}
df_importances_logreg = {}
df_markers_rank_significance_logreg = {}

for ctj in ['B cell', 'T cell']:#u_cell_types:
    if  (adata.obs[label_key]==ctj).sum() < 30:
        print(f"Not enough cells from class {ctj}")
        continue

    # get markers
    if markers == "diffexpr":
        if precomputed_ctj_markers is not None:
            ctj_marker_genes_precomputed = df_ctj_marker_genes_precomputed[ctj]

        sc.tl.rank_genes_groups(adata, groupby=label_key, groups=[ctj], reference="rest", method="wilcoxon")
        ctj_marker_genes = sc.get.rank_genes_groups_df(adata, group=ctj)["names"][: top_n].tolist()
    else:
        ctj_marker_genes = df_markers[df_markers["cell_type"] == ctj]["gene"].tolist()
        ctj_marker_genes = [g for g in ctj_marker_genes if g in adata.var_names]

        assert len(ctj_marker_genes), f"no markers found for {ctj}"



    df_ctj_marker_genes[ctj] = ctj_marker_genes

    for cti in  ['B cell', 'T cell']:#u_cell_types:
        if cti == ctj:
            continue
        print(cti, ctj)

        adata.obs[f"has_{ctj}_neighbor"] = knnlabels[ctj]>0

        # Filter for cti
        adata_cti = adata[adata.obs[label_key] == cti]

        if (   (adata_cti.obs[f"has_{ctj}_neighbor"]).sum() < 30
            or (~adata_cti.obs[f"has_{ctj}_neighbor"]).sum() < 30
            ):
            print(f"Not enough cells from each class to test {cti} with {ctj} neighbors")
            continue


        ###
        ### DIFF EXPR TEST: check DE genes between cti with ctj neighbor or not
        ###
        # adata_cti.obs[f'has_{ctj}_neighbor_str'] = adata_cti.obs[f'has_{ctj}_neighbor'].astype(str)
        # sc.tl.rank_genes_groups(adata_cti, groupby=f"has_{ctj}_neighbor_str", groups=['True'], reference='False', method="wilcoxon")
        # df_diffexpr[cti,ctj] = sc.get.rank_genes_groups_df(adata_cti, group='True').sort_values('pvals_adj')


        # # get significance from gsea and hypergeometric test
        # df_markers_rank_significance_diffexpr[cti, ctj] = _utils.get_marker_rank_significance(
        #     rnk=df_diffexpr[cti, ctj].set_index("names")["logfoldchanges"],
        #     gene_set=ctj_marker_genes,
        #     top_n=top_n,
        # )



        ####
        #### LOGISTIC REGRESSION TEST: predict ctj in kNN based on cti expression
        ####

        # train logreg model
        df_permutations_logreg[cti, ctj], df_importances_logreg[cti, ctj] = _utils.logreg(
            X=adata_cti.X,  # adata_cti.layers["X_normalised"],
            y=adata_cti.obs[f"has_{ctj}_neighbor"],
            feature_names=adata.var_names,
            scoring=scoring,
            test_size=0.2,
            n_splits=n_splits,
            n_permutations=n_permutations,
            n_repeats=n_repeats,
            random_state=0,
            max_iter=500,
            importance_mode="coef",
            class_weight="balanced",
            cv_mode=cv_mode,
            spatial_coords=adata_cti.obsm["spatial"],
        )

        # get significance from gsea and hypergeometric test
        rank_metric = "importances"
        df_markers_rank_significance_logreg[cti, ctj] = pd.DataFrame(index=index_diffexpr_metrics)
        dict_ctj_marker_genes = {"": ctj_marker_genes}

        if precomputed_ctj_markers is not None:
            # also compute scores for precomputed marker gene list
            dict_ctj_marker_genes["_precomputed"] = ctj_marker_genes_precomputed

        for n in list_n_markers:
            for k_, markers_ in dict_ctj_marker_genes.items():
                markers_n_ = markers_[:n]

                df_markers_rank_significance_logreg[cti, ctj][rank_metric + k_ + f"_{n=}"] = (
                    _utils.get_marker_rank_significance(
                        rnk=df_importances_logreg[cti, ctj][rank_metric].sort_values(ascending=False),
                        gene_set=markers_n_,
                        top_n=top_n,
                    ).iloc[0]
                )

INFO     reading                                                                                                   
         /work/PRTNR/CHUV/DIR/rgottar1/spatial/env/xenium_paper/data/xenium/processed/segmentation/10x_15um/breast/
         breast/1GUC/1GUC/normalised_results/outs/cell_feature_matrix.h5                                           


/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/spatialdata/_core/spatialdata.py:185: UserWarning: The table is annotating 'cell_labels', which is not present in the SpatialData object.
  self.validate_table_in_spatialdata(v)
/tmp/ipykernel_901051/568198059.py:97: ImplicitModificationWarning: Setting element `.layers['X_normalised']` of view, initializing view as actual.
  adata.layers['X_normalised'] = X_normalised
/work/PRTNR/CHUV/DIR/rgottar1/spatial/env/jbac/miniforge3/envs/spatial/lib/python3.11/site-packages/scanpy/preprocessing/_normalization.py:207: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


T cell B cell


ValueError: This solver needs samples of at least 2 classes in the data, but the data contains only one class: False